# Advanced CRUD
### Here will learn hands-on advanced CRUD operation 


**Find with Regex**

```javascript
// Find names starting with "A"
db.users.find({ name: { $regex: /^A/ } })
Find with Pagination
javascript
// Skip and limit for pagination
db.users.find().skip(10).limit(5)
Update with Array
javascript
// Push to array
db.users.updateOne({ name: "Alice" }, { $push: { hobbies: "reading" } })

// Pop from array
db.users.updateOne({ name: "Alice" }, { $pop: { hobbies: -1 } }) // remove first
Replace
javascript
// Replace entire document
db.users.replaceOne({ name: "Alice" }, { name: "Alice", age: 32, city: "New York" })
```

## Embedded Documents
**Example 1:** Simple Embedded Document

```javascript
// Insert a user with an embedded address
db.users.insertOne({
  name: "John",
  address: {
    street: "123 Main St",
    city: "Springfield",
    state: "IL"
  }
})
```

**Example 2:** Query on Embedded Document

```javascript
// Find by city
db.users.find({ "address.city": "Springfield" })
Example 3: Update Embedded Document
javascript
// Update the city of John's address
db.users.updateOne(
  { name: "John" },
  { $set: { "address.city": "Chicago" } }
)
```
**Example 4:** Embedded Array of Documents

```javascript
// User with multiple addresses
db.users.insertOne({
  name: "Jane",
  addresses: [
    { type: "home", street: "123 Main St", city: "Springfield" },
    { type: "work", street: "456 Oak St", city: "Chicago" }
  ]
})
```
**Example 5:** Query Embedded Array

```javascript
// Find users with a work address in Chicago
db.users.find({ "addresses.type": "work", "addresses.city": "Chicago" })
```

**Example 6:** Update Specific Element in Array

```javascript
// Update Jane's work address city
db.users.updateOne(
  { name: "Jane", "addresses.type": "work" },
  { $set: { "addresses.$.city": "New York" } }
)
```
**Explanation:** The $ operator refers to the matched element in the array.

## Referenced Documents
**Example 1:** Separate Collections

```javascript
// Authors collection
db.authors.insertOne({ _id: 1, name: "J.K. Rowling" })

// Books collection with reference to author
db.books.insertOne({
  title: "Harry Potter",
  authorId: 1
})
```

**Example 2:** Lookup to Join

```javascript
// Aggregate to join books with authors
db.books.aggregate([
  {
    $lookup: {
      from: "authors",
      localField: "authorId",
      foreignField: "_id",
      as: "author"
    }
  }
])
```

**Example 3:** Multiple References

```javascript
// Books with multiple authors (array of references)
db.books.insertOne({
  title: "Advanced MongoDB",
  authorIds: [1, 2]
})

// Then lookup to get all authors
db.books.aggregate([
  {
    $lookup: {
      from: "authors",
      localField: "authorIds",
      foreignField: "_id",
      as: "authors"
    }
  }
])
```

**Example 4:** Denormalization with References
Sometimes we denormalize for performance.

```javascript
// Instead of only storing the authorId, we store some author data too
db.books.insertOne({
  title: "Harry Potter",
  authorId: 1,
  authorName: "J.K. Rowling"
})

// This avoids a join for the author name, but we have to update if the author changes name.
```
## Aggregation Pipeline
*We'll build up aggregation examples.*

**Example 1:** Simple Grouping

```javascript
// Group by category and count
db.products.aggregate([
  { $group: { _id: "$category", count: { $sum: 1 } } }
])
Example 2: Group with Average
javascript
// Average price by category
db.products.aggregate([
  { $group: { _id: "$category", avgPrice: { $avg: "$price" } } }
])
```

**Example 3:** Multiple Grouping Fields

```javascript
// Group by category and brand
db.products.aggregate([
  { $group: { _id: { category: "$category", brand: "$brand" }, count: { $sum: 1 } } }
])
```
**Example 4:** Project and Group

```javascript
// Project first then group
db.products.aggregate([
  { $project: { category: 1, price: 1, discountPrice: { $multiply: ["$price", 0.9] } } },
  { $group: { _id: "$category", avgDiscountPrice: { $avg: "$discountPrice" } } }
])
```

**Example 5:** Unwind and Group

```javascript
// Products with an array of tags
db.products.aggregate([
  { $unwind: "$tags" },
  { $group: { _id: "$tags", count: { $sum: 1 } } }
])
```
**Example 6:** Lookup and Then Group

```javascript
// Orders with product details
db.orders.aggregate([
  {
    $lookup: {
      from: "products",
      localField: "productId",
      foreignField: "_id",
      as: "product"
    }
  },
  { $unwind: "$product" },
  { $group: { _id: "$product.category", totalSales: { $sum: "$amount" } } }
])
```

**Example 7:** Bucket for Histograms

```javascript
// Group prices into buckets
db.products.aggregate([
  {
    $bucket: {
      groupBy: "$price",
      boundaries: [0, 100, 500, 1000, 2000],
      default: "Above2000",
      output: {
        count: { $sum: 1 },
        products: { $push: "$name" }
      }
    }
  }
])
```

**Example 8:** Facet for Multiple Aggregations

```javascript
// Multiple aggregations in one pipeline
db.products.aggregate([
  {
    $facet: {
      "priceStats": [
        { $group: { _id: null, avgPrice: { $avg: "$price" }, maxPrice: { $max: "$price" } } }
      ],
      "categoryCounts": [
        { $group: { _id: "$category", count: { $sum: 1 } } }
      ]
    }
  }
])
```

**Example 9:** Conditional Grouping

```javascript
// Group by price range (using $switch)
db.products.aggregate([
  {
    $project: {
      name: 1,
      price: 1,
      priceRange: {
        $switch: {
          branches: [
            { case: { $lt: ["$price", 100] }, then: "Cheap" },
            { case: { $lt: ["$price", 500] }, then: "Mid" },
            { case: { $lt: ["$price", 1000] }, then: "Expensive" }
          ],
          default: "Very Expensive"
        }
      }
    }
  },
  {
    $group: {
      _id: "$priceRange",
      count: { $sum: 1 }
    }
  }
])
```

**Example 10:** Date Aggregation

```javascript
// Group by year and month of sale
db.orders.aggregate([
  {
    $group: {
      _id: {
        year: { $year: "$saleDate" },
        month: { $month: "$saleDate" }
      },
      total: { $sum: "$amount" }
    }
  },
  { $sort: { "_id.year": 1, "_id.month": 1 } }
])
```


In [ ]:
// app.js
const { MongoClient } = require('mongodb');

// Connection URI
const uri = "mongodb://localhost:27017"; // or Atlas connection string

// Create a new MongoClient
const client = new MongoClient(uri);

async function main() {
  try {
    // Connect to the MongoDB cluster
    await client.connect();
    console.log("Connected to MongoDB");

    // Reference to the database
    const database = client.db('myblog');

    // Reference to the collection
    const posts = database.collection('posts');

    // Example: Insert a document
    const result = await posts.insertOne({
      title: "Node.js and MongoDB",
      content: "How to use MongoDB with Node.js",
      date: new Date()
    });
    console.log(`Inserted document with _id: ${result.insertedId}`);

  } finally {
    // Close the connection
    await client.close();
  }
}

main().catch(console.error);